<a href="https://colab.research.google.com/github/mashyko/NLP_Mecab/blob/master/fasttext_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text classification: Fasttextを用いたテキスト分類

# Fasttextのインストール

コマンド　./fasttext は機能しないので、使用しません。


In [2]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!make
# for python bindings :
!pip install fasttext

fatal: destination path 'fastText' already exists and is not an empty directory.
make: *** No targets specified and no makefile found.  Stop.


In [3]:
import fasttext
help(fasttext.FastText)

Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    eprint(cls, *args, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You migh

Fasttextのチュートリアルに従って、cooking.stackexchangeのデータを読み込む（英文です
）
日本語と異なり、英文は分析がシンプルです。


In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt

--2020-01-03 01:38:02--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457609 (447K) [application/x-tar]
Saving to: ‘cooking.stackexchange.tar.gz’

cooking.stackexchan 100%[===================>] 446.88K   675KB/s    in 0.7s    

2020-01-03 01:38:04 (675 KB/s) - ‘cooking.stackexchange.tar.gz’ saved [457609/457609]

cooking.stackexchange.id
cooking.stackexchange.txt
readme.txt
__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaur

In [5]:
!wc cooking.stackexchange.txt

  15404  169582 1401900 cooking.stackexchange.txt


datasetは 15404 examplesから構成されます。 これを12404 examplesのtraining setと 3000 examplesのvalidation setに分割します。


In [0]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

単純なclassifierを作成：label付きなので、train_supervised を使用します。labelがないときは、train_unsupervisedを使います。

In [0]:
model = fasttext.train_supervised(input="cooking.train")

In [0]:
model.save_model("model_cooking.bin")

モデルを検証します


In [9]:
model.predict("Which baking dish is best to bake a banana bread ?")

(('__label__baking',), array([0.0723336]))

In [10]:
model.predict("Why not put knives in the dishwasher?", k=5)

(('__label__food-safety',
  '__label__baking',
  '__label__bread',
  '__label__substitutions',
  '__label__equipment'),
 array([0.07458788, 0.0736473 , 0.0439545 , 0.03730712, 0.03408002]))

In [11]:
model.test("cooking.valid")

(3000, 0.13533333333333333, 0.05852674066599395)

少し複雑なclassifierを作成

In [12]:
model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

(('__label__baking', '__label__bananas', '__label__bread'),
 array([1.00001001, 0.9603712 , 0.64423513]))

Word representations

English Wikipediaの一部を使用します。データをMatt Mahoney's websiteからダウンロードします。

In [22]:
!cd fastText/
!mkdir data
!wget -c http://mattmahoney.net/dc/enwik9.zip -P data
!unzip data/enwik9.zip -d data

mkdir: cannot create directory ‘data’: File exists
--2020-01-03 01:57:09--  http://mattmahoney.net/dc/enwik9.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  data/enwik9.zip
  inflating: data/enwik9             


In [23]:
!perl fastText/wikifil.pl data/enwik9 > data/fil9
!ls
!head -c 80 data/fil9

cooking.stackexchange.id      cooking.train  fastText		result
cooking.stackexchange.tar.gz  cooking.valid  model_cooking.bin	sample_data
cooking.stackexchange.txt     data	     readme.txt
 anarchism originated as a term of abuse first used against early working class 

Python API の fasttext.train_unsupervised が正常に作動しないようです。理由はわかりません。Commandline を用いた方が無難です。


fastText の　makeコマンドが有効ではないので、以下の !./fasttext コマンドラインを用いた処理の多くはこのColabでは実行できません。ダウンロードして、各自のPCで利用ください。

In [24]:
! mkdir result
! ./fasttext skipgram -input data/fil9 -output result/fil9


mkdir: cannot create directory ‘result’: File exists
/bin/bash: ./fasttext: No such file or directory


In [16]:
! ls -l result


total 0


In [17]:
! head -n 4 result/fil9.vec


head: cannot open 'result/fil9.vec' for reading: No such file or directory


In [18]:
! echo "asparagus pidgey yellow" | ./fasttext print-word-vectors result/fil9.bin


/bin/bash: ./fasttext: No such file or directory


In [19]:
!./fasttext nn result/fil9.bin

Query word?

/bin/bash: ./fasttext: No such file or directory
Object `word` not found.


Word analogies

In [20]:
!./fasttext analogies result/fil9.bin

Query triplet (A - B + C)? berlin germany france


SyntaxError: ignored

Mecabのインストール


In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

日本語wikiの文書ファイルを使用する。

単語の分散表現を学習させる際、Wikipedia 全体のサイズが必要ないときに使われるコーパスとして text8 があります。 text8 は、Wikipedia に対してクリーニング等の処理をした後、100MB分切り出して作成されています。 text8 は前処理済みで簡単に使えるので、チュートリアル等でよく利用されています。日本語バージョンのtext8コーパスをダウンロードします。

In [0]:
!wget -c https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip -P data

In [0]:
!./fasttext skipgram -input data/ja.text8 -output result/ja.text8_model

In [0]:
! head -n 4 result/fil9.vec

In [0]:
!./fasttext nn ~/NLP/corpora/ja.text8_model.bin

Query word? 利根川

元荒川 0.844393
鳴瀬川 0.832339
綾瀬川 0.830789
常盤川 0.807554
永野川 0.807308
狩野川 0.807213
用水路 0.80265
吉野川 0.798709
品井沼 0.795811
大野川 0.792659


以上